In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
# disable warnings
import warnings
warnings.filterwarnings('ignore')

# params 1
nodes = ['1', '2', '3', '4', '5']
send_edges = [('1', '2', 'a'),('2', '3', 'b'),('3', '4', 'c'),('5', '4', 'd')]
receive_edges = [('3', '2', 'a'),('4', '3', 'b'),('4', '5', 'd')]


# # params 2
# nodes = ['1', '2', '3', '4', '5', '6', '7', '8', '9']
# send_edges = [
#     ('1', '2', 'a'),
#     ('2', '3', 'b'),
#     ('4', '5', 'c'),
#     ('5', '8', 'd'),
#     ('5', '8', 'e'),
#     ('5', '8', 'f')]
# receive_edges = [('2', '4', 'b'),
#                  ('4', '6', 'd'),
#                  ('4', '7', 'e'),
#                  ('6', '8', 'd'),
#                  ('7', '8', 'e'),
#                  ('8', '9', 'a'),
#                  ]

In [2]:
# first: map sequance of nodes (note: no internal loops needed)
all_edges = send_edges + receive_edges
nodes_tier = []
passed_nodes = []
left_nodes = nodes.copy()
node_tier = 0
while len(passed_nodes) < len(nodes):
    nodes_tier.append([])
    for node in left_nodes:
        incoming_edges = [edge for edge in all_edges if edge[1] == node]
        # if incoming_edges subgroup in passed_nodes then add to nodes_tier[node_tier]
        if len([edge for edge in incoming_edges if edge[0] in passed_nodes]) == len(incoming_edges):
            nodes_tier[node_tier] = nodes_tier[node_tier] + [node]
    passed_nodes = passed_nodes + nodes_tier[node_tier]
    left_nodes = [node for node in nodes if node not in passed_nodes]
    node_tier += 1

print(nodes_tier)

[['1'], ['2'], ['3', '4'], ['5']]


In [3]:
# have a tuple for each node, init node: (incoming_str, outgoing_str_to_edge, remaining_str)
node_exptree = {}
node_strings = {}
for node in nodes:
    node_exptree[node] = [{}, {}, {}] # incoming_str, outgoing_str_to_edge, outgoing_str_to_edge joined, remaining_str_with_incoming
    node_strings[node] = ['', {}, '', ''] # incoming_str, outgoing_str_to_edge, outgoing_str_to_edge joined, remaining_str_with_incoming

#
while len(nodes_tier) > 0:
    for node in nodes_tier[0]:
        incoming_edges = [edge for edge in all_edges if edge[1] == node]
        incoming_nodes = np.unique([edge[0] for edge in incoming_edges])
        outgoing_edges = [edge for edge in all_edges if edge[0] == node]
        # 
        outgoing_sending_edges = [edge for edge in outgoing_edges if edge in send_edges]
        outgoing_receive_edges = [edge for edge in outgoing_edges if edge in receive_edges]
        # first, calculate incoming_str
        if len(incoming_edges) == 0:
            node_strings[node][0] = 'inf'
            node_exptree[node][0] = {
                'exp' : 'inf',
                'value': 'inf'
            }
        else:
            for in_node in incoming_nodes:
                if node_strings[node][0] == '':
                    # node_exptree[node][0] = node_exptree[edge[0]][1][node]
                    node_strings[node][0] = node_strings[in_node][1][node]
                    node_exptree[node][0] = node_exptree[in_node][1]['sum'][node]
                else:
                    # node_exptree[node][0] += " + " + node_exptree[edge[0]][1][node]
                    node_strings[node][0] += " + " + node_strings[in_node][1][node]
                    node_exptree[node][0] = {
                        'exp' : 'add',
                        'left': node_exptree[node][0],
                        'right': node_exptree[in_node][1]['sum'][node]
                    }
        # second, calculate outgoing_str_to_edge for each outgoing edge
        if len(outgoing_edges) > 0:
            # create a list of tuples (node, outgoing_str_to_edge)
            outgoing_str_to_edge = {}
            outgoing_exptree_to_edge = {}
            for edge in outgoing_sending_edges:
                if edge[1] in outgoing_str_to_edge:
                    # outgoing_str_to_edge[edge[1]] += f" + |w|_{edge[2]}"
                    outgoing_str_to_edge[edge[1]] += f" + |w|_{edge[2]}"
                    outgoing_exptree_to_edge[edge[1]] = {
                        'exp' : 'add',
                        'left': outgoing_exptree_to_edge[edge[1]],
                        'right': {
                            'exp': 'count',
                            'letter': edge[2]
                        }
                    }
                else:
                    # outgoing_str_to_edge[edge[1]] = f"|w|_{edge[2]}"
                    outgoing_str_to_edge[edge[1]] = f"|w|_{edge[2]}"
                    outgoing_exptree_to_edge[edge[1]] = {
                        'exp': 'count',
                        'letter': edge[2]
                    }
            # find outgoing_str_to_edge for each outgoing receiving edge
            ouput_str = node_strings[node][0]
            ouput_exptree = node_exptree[node][0]
            for edge in outgoing_sending_edges:
                ouput_str += f" - {outgoing_str_to_edge[edge[1]]}"
                ouput_exptree = {
                    'exp' : 'sub',
                    'left': ouput_exptree,
                    'right': outgoing_exptree_to_edge[edge[1]]
                }
                # ouput_str += f" - |w|_{edge[2]}"
            # if only one outgoing receive edge, then (input - senders_output)_t_{l}
            if len(outgoing_receive_edges) == 1:
                # add to output -1 if a sending is in outgoing_receive_edges
                if outgoing_receive_edges[0][2] in [edge[2] for edge in outgoing_sending_edges]:
                    ouput_str += f" - 1"
                    ouput_exptree = {
                        'exp' : 'sub',
                        'left': ouput_exptree,
                        'right': {
                            'exp': 'num',
                            'num': 1
                        }
                    }
                # outgoing_str_to_edge[outgoing_receive_edges[0][1]] = f"({ouput_str})_to_{outgoing_receive_edges[0][2]}"
                outgoing_str_to_edge[outgoing_receive_edges[0][1]] = f"({ouput_str})_to_{outgoing_receive_edges[0][2]}"
                outgoing_exptree_to_edge[outgoing_receive_edges[0][1]] = {
                    'exp' : 'to',
                    'inner': ouput_exptree,
                    'to': outgoing_receive_edges[0][2]
                }
            # if many outgoing receive edges, then (input - senders_output)_section_{l}_{all_receive_letters}
            elif len(outgoing_receive_edges) > 1:
                # join all letters in outgoing_receive_edges to one string named letters
                letters = "".join([edge[2] for edge in outgoing_receive_edges])
                for edge in outgoing_receive_edges:
                    cur_ouput_str = ouput_str
                    cur_ouput_exptree = ouput_exptree
                    # add to output -1 if a sending is in outgoing_receive_edges
                    if edge[2] in [e[2] for e in outgoing_sending_edges]:
                        cur_ouput_str += f" - 1"
                        cur_ouput_exptree = {
                            'exp' : 'sub',
                            'left': cur_ouput_exptree,
                            'right': {
                                'exp': 'num',
                                'num': 1
                            }
                        }
                    #
                    if edge[1] in outgoing_str_to_edge:
                        # outgoing_str_to_edge[edge[1]] += f" + ({ouput_str})_section_{edge[2]}_({letters})"
                        outgoing_str_to_edge[edge[1]] += f" + ({cur_ouput_str})_section_{edge[2]}_({letters})"
                        outgoing_exptree_to_edge[edge[1]] = {
                            'exp' : 'add',
                            'left': outgoing_exptree_to_edge[edge[1]],
                            'right': {
                                'exp': 'section',
                                'inner': cur_ouput_exptree,
                                'section': edge[2],
                                'letters': letters
                            }
                        }
                    else:
                        # outgoing_str_to_edge[edge[1]] = f"({ouput_str})_section_{edge[2]}_({letters})"
                        outgoing_str_to_edge[edge[1]] = f"({cur_ouput_str})_section_{edge[2]}_({letters})"
                        outgoing_exptree_to_edge[edge[1]] = {
                            'exp': 'section',
                            'inner': cur_ouput_exptree,
                            'section': edge[2],
                            'letters': letters
                        }
            # add to node_strings
            node_strings[node][1] = outgoing_str_to_edge
            # add to node_exptree as array
            node_exptree[node][1] = {
                'exp': 'sum',
                'sum': outgoing_exptree_to_edge
            }
            # node_exptree[node][2] as joined outgoing_str_to_edge
            node_strings[node][2] = " + ".join([outgoing_str_to_edge[key] for key in outgoing_str_to_edge])
        # third, calculate remaining_str, simply (input - senders_output)
        node_strings[node][3] = f"{node_strings[node][0]} - ({node_strings[node][2]})" if node_exptree[node][0] != 'inf' else 'inf'
        node_exptree[node][2] = {
            'exp' : 'sub',
            'left': node_exptree[node][0],
            'right': node_exptree[node][1]
        }
    nodes_tier = nodes_tier[1:]

# add all to dataframe
df = pd.DataFrame(columns=['node', 'incoming', 'outgoing', 'remaining'])
for node in nodes:
    df = df.append({'node': node, 'incoming': node_strings[node][0], 'outgoing': node_strings[node][2], 'remaining': node_strings[node][3]}, ignore_index=True)

print(df.to_string())

# # print object in json format
# print(json.dumps(node_exptree, indent=4))


  node              incoming                      outgoing                               remaining
0    1                   inf                         |w|_a                           inf - (|w|_a)
1    2                 |w|_a  |w|_b + (|w|_a - |w|_b)_to_c  |w|_a - (|w|_b + (|w|_a - |w|_b)_to_c)
2    3  (|w|_a - |w|_b)_to_c                         |w|_d          (|w|_a - |w|_b)_to_c - (|w|_d)
3    4                 |w|_b                         |w|_c                         |w|_b - (|w|_c)
4    5         |w|_d + |w|_c                                                    |w|_d + |w|_c - ()


In [4]:
def calculate_sum_of_letter(word, letter):
    return len([l for l in word if l == letter])

def cut_word_to_letter(word, letter):
    # get last index of letter in word
    last_index = word.rfind(letter)
    # if letter not in word, return empty string
    if last_index == -1:
        return ''
    # else return until last index
    return word[:last_index]

def calculate_sections(word, letters):
    # cut word to sections by letters
    sections = []
    prev_section = ''
    working_section = ''
    for l in word:
        if l in letters:
            sections.append((l, prev_section, working_section))
            prev_section = working_section
        working_section = working_section + l
    # return merged sections
    return sections

def eval_exptree(exp, word):
    # print(exp)
    # print(word)
    if len(word) == 0:
        return 0
    if exp:
        if exp['exp'] == 'inf':
            return float('inf')
        elif exp['exp'] == 'add':
            return eval_exptree(exp['left'], word) + eval_exptree(exp['right'], word)
        elif exp['exp'] == 'sub':
            return eval_exptree(exp['left'], word) - eval_exptree(exp['right'], word)
        elif exp['exp'] == 'to':
            return eval_exptree(exp['inner'], cut_word_to_letter(word, exp['to']))
        elif exp['exp'] == 'section':
            sections = calculate_sections(word, exp['letters']) 
            letter_sections = [section for section in sections if section[0] == exp['section']]
            sum_current_sections = sum([eval_exptree(exp['inner'], section[2]) for section in letter_sections])
            sum_prev_sections = sum([eval_exptree(exp['inner'], section[1]) for section in letter_sections])
            return sum_current_sections - sum_prev_sections
            # eval_exptree(exp['inner'], calculate_section(word, exp['section'], exp['letters']))
        elif exp['exp'] == 'sum':
            return sum([eval_exptree(exp['sum'][key], word) for key in exp['sum']])
        elif exp['exp'] == 'num':
            return exp['num']
        elif exp['exp'] == 'count':
            return calculate_sum_of_letter(word, exp['letter'])
        else:
            raise Exception('Unknown expression type ' + exp['exp'])
    else:
        return 0

def verifier(word, node_exptree):
    # calculate for each node number based on cur_word and remaining_str
    node_values = {}
    for node in node_exptree:
        node_values[node] = eval_exptree(node_exptree[node][2], word)
    return node_values

def debug_exp(exp, word):
    if len(word) == 0:
        return '~0'
    if exp:
        # print(exp)
        if exp['exp'] == 'inf':
            return 'inf'
        elif exp['exp'] == 'add':
            return f"({debug_exp(exp['left'], word)} + {debug_exp(exp['right'], word)})"
        elif exp['exp'] == 'sub':
            return f"({debug_exp(exp['left'], word)} - {debug_exp(exp['right'], word)})"
        elif exp['exp'] == 'to':
            return f"{debug_exp(exp['inner'], cut_word_to_letter(word, exp['to']))}"
        elif exp['exp'] == 'section':
            return f"{debug_exp(exp['inner'], calculate_section(word, exp['section'], exp['letters']))}"
        elif exp['exp'] == 'sum':
            return f"({' + '.join([debug_exp(exp['sum'][key], word) for key in exp['sum']])})"
        elif exp['exp'] == 'num':
            return str(exp['num'])
        elif exp['exp'] == 'count':
            return f"{calculate_sum_of_letter(word, exp['letter'])}"
        else:
            raise Exception('Unknown expression type ' + exp['exp'])
    else:
        return '0'
    
    
# print(verifier('aaaabcdaaab', node_exptree))
# print(debug_exp(node_exptree['9'][2], 'aaaabcdaaab'))
# print(verifier('aaaabcdaaab', node_exptree)['4'])
# print(json.dumps(node_exptree['9'][2], indent=4))

In [5]:

def send_edge(letter, node_values, send_edges, receive_edges):
    # first, deal with sending edge
    for edge in send_edges:
        # if letter in edge, add to out
        if letter == edge[2]:
            if node_values[edge[0]][0] == 0:
                return 0
            node_values[edge[0]][0] -= 1 # deal with initial not out since sending overrules receiving
            node_values[edge[1]][1] += 1
            break
    # second, deal with receiving edges
    for edge in receive_edges:
        # if letter in edge, add to in
        if letter == edge[2]:
            node_values[edge[0]][2] = node_values[edge[0]][0]
            node_values[edge[1]][1] += node_values[edge[0]][0]
    # commit all in and out
    for node in nodes:
        node_values[node][0] = node_values[node][0] + node_values[node][1] - node_values[node][2]
        node_values[node][1] = 0
        node_values[node][2] = 0
    return 1

# simulate word over the BP
def simulate(word, nodes, send_edges, receive_edges):
    # init node_values
    node_values = {}
    for node in nodes:
        node_values[node] = [0, 0, 0] # [initial, in, out]
    node_values[nodes[0]] = [float('inf'), 0, 0]
    # for each letter in word
    for letter in word:
        # send letter
        if(send_edge(letter, node_values, send_edges, receive_edges) == 0):
            raise Exception('Node ' + edge[0] + ' has no cores')
    # return node_values first value in each node
    # convert to [(node, node_values[node][0]) for node in nodes] to dict
    return dict([(node, node_values[node][0]) for node in nodes])

In [6]:
def compare_outputs(word, nodes, send_edges, receive_edges):
    #
    
    # get verifier output
    verifier_output = verifier(word, node_exptree)
    # get simulate output
    simulate_output = simulate(word, nodes, send_edges, receive_edges)
    # compare outputs
    for node in nodes:
        if verifier_output[node] != simulate_output[node]:
            raise Exception(f"Node {node} verifier output {verifier_output[node]} != simulate output {simulate_output[node]}")
    print('Y')

In [7]:
# compare_outputs('aaaabcdaaab', nodes, send_edges, receive_edges)

In [8]:
def gen_word_run(word, node_values, send_edges, receive_edges):
    # search for non empty nodes, and rand some number of sending actions from the node
    non_empty_nodes = [node for node in nodes if node_values[node][0] > 0]
    # 
    possible_send_edges = [edge for edge in send_edges if edge[0] in non_empty_nodes]
    all_possible_letters = [edge[2] for edge in possible_send_edges]
    num_of_instances = [calculate_sum_of_letter(word, letter) for letter in all_possible_letters]
    # for each possible letter, give a nuumber
    letter_prob_numerator = [max(sum(num_of_instances) - num_of_instances[i], 1) for i in range(len(all_possible_letters))]
    letter_prob = [letter_prob_numerator[i] / sum(letter_prob_numerator) for i in range(len(letter_prob_numerator))]
    letter = np.random.choice(all_possible_letters, p=letter_prob)
    # get node of letter
    node = [edge[0] for edge in possible_send_edges if edge[2] == letter][0]
    #
    # node = np.random.choice(non_empty_nodes)
    # select number of sending actions, if float('inf') rand up to 10
    max_num_sending_actions = 10 if node_values[node][0] == float('inf') else node_values[node][0]
    # number of sending actions, randomized with favor to lower numbers
    num_sending_actions = np.random.randint(0, max_num_sending_actions)
    # select letters to send
    node_send_edges = [edge[2] for edge in send_edges if edge[0] == node]
    if len(node_send_edges) == 0:
        return ''
    letter = np.random.choice(node_send_edges)
    # send letters
    for i in range(num_sending_actions):
        if(send_edge(letter, node_values, send_edges, receive_edges) == 0):
            break
        word += letter
    return word

def gen_word(send_edges, receive_edges, max_runs):
    # simulate word over the BP
    node_values = {}
    for node in nodes:
        node_values[node] = [0, 0, 0] # [initial, in, out]
    node_values[nodes[0]] = [float('inf'), 0, 0]
    # set random number of runs out of max_runs
    num_of_runs = max_runs#np.random.randint(1, max_runs)
    # run gen_word_run num_of_runs times
    word = ''
    for i in range(num_of_runs):
        word = gen_word_run(word, node_values, send_edges, receive_edges)
    return word


In [9]:
# gen n words and compare outputs
for i in range(100):
    word = gen_word(send_edges, receive_edges, 25)
    print(f"Word {i}: {word}")
    compare_outputs(word, nodes, send_edges, receive_edges)

Word 0: aabaaaaaabbbbbbaaaaaaaccccccddddddaaaaaaaaabbbbaaaaaabaaaaaaa
Y
Word 1: aabaaaabbbbaaaaaccdddddccaaaaaaa
Y
Word 2: aaaaaaaabbbbbbbccccccaaaaaaaaaaaaa
Y
Word 3: aaaaaaaaaaaaabbbbbbbaaaaccccccddddddddaaaaadaaaaabbbbbccc
Y
Word 4: aaaabbcdaaaaaaaaaabbbbbbbcccccccdddaaaaaaa
Y
Word 5: aaaaaaaabbcddddaaaaaaaaaaaaaadbbbbbbbbbbbbccccccccccccaaaaaddbbbbc
Y
Word 6: aaaaaaaaabbaaabbbbbbcccccccaaaaaaaaadddaaaaaaaaaaabbbbccddddddddddd
Y
Word 7: aaaaaabbcaaadddaaaaaaaabbbbbbbbbcccccddcccc
Y
Word 8: aaaaaaabbbaaaaaccdddddddaaaaaaaaaaaaaaaaaaaaaaaaabbbbbbbbbbbbbbbbbbbbbbdcccccccccccc
Y
Word 9: aaabbcaaaaaaaaabbbbbbbbcccccaaaacccaa
Y
Word 10: aaaaaaaabbbbcccdddaaaaaaabbbbbbccccccdaaaaaabbbbbaa
Y
Word 11: aaaaabbbbaaaaaaaaaaaaaaaaacccdddddddddddddddaaaabdcaaaaabbbcccd
Y
Word 12: aaaaaaaaaabbbbcccdddddaaaaaaaaaaaaaaaaa
Y
Word 13: aabaaaaaaabbbbbaaaaaaaaabbbbbbbbc
Y
Word 14: aaabbaaaaaaaaacdddddddddaaaaaabb
Y
Word 15: aaaaaaabbbbbbcccccaaaabbbcadaaaa
Y
Word 16: aaaaaaabbbbbccccdaab
Y
Word 17: aaab

In [10]:
w = 'aaaa'

verifier(w, node_exptree)

{'1': inf, '2': 4, '3': 0, '4': 0, '5': 0}

In [11]:
simulate(w, nodes, send_edges, receive_edges)

{'1': inf, '2': 4, '3': 0, '4': 0, '5': 0}

In [12]:
compare_outputs(w, nodes, send_edges, receive_edges)

Y
